In [1]:
#!{sys.executable} -m pip install --upgrade --no-cache-dir pysimplegui
#Importing the data extractor scripts
import sys
sys.path.insert(0, 'C:\\Users\\AU451FE\\OneDrive - EY\\Desktop\\Python\\HSreplay_scraper\\Scripts')

import DataProcessor as P
#import Extractors
#from UltimateExtractor import UltimateExtractor as UE

#Other useful packages
import time
import datetime
import pandas as pd
import numpy as np
import re #String search
import os
import PySimpleGUI as sg

In [2]:
#driver_path = 'C:\Users\AU451FE\OneDrive - EY\Desktop\Python\HSreplay_scraper\chromedriver'
#deck_folder = 'C:\Users\AU451FE\OneDrive - EY\Desktop\Python\HSreplay_scraper\Data Frames'
#analysis_path = 'C:\Users\AU451FE\OneDrive - EY\Desktop\Python\HSreplay_scraper\Data Frames\Analyzed'

driver_path = r'C:\Users\hso20\Python\HSreplay_scraper\chromedriver'
deck_folder = r'C:\Users\hso20\Python\HSreplay_scraper\Data Frames'
analysis_path = r'C:\Users\hso20\Python\HSreplay_scraper\Data Frames\Analyzed'

In [3]:
filename = sg.popup_get_file('Enter the file you wish to process')
sg.popup('You entered', filename)

'OK'

In [4]:
foldername = sg.popup_get_folder('Enter the folder you wish to process')
sg.popup('You entered', foldername)

'OK'

In [35]:
import PySimpleGUI as sg
class GraphicalArchmage:
    '''Docstring here
    '''
    def __init__(self):
        pass

    def open_init_w(self):
        #Text
        col1 = sg.Column([[sg.Frame('', [[sg.Text('\nWelcome to the Hearthstone Archmage\n', size = (60,3),
                                        font = ('Courier, 35'), justification = 'center')]])]], justification = 'center')
        col2 = sg.Column([[sg.Frame('', [[sg.Text('Specifiy the type of activity\n you wish to do', size = (60,2),
                                        font = ('Courier, 20'), justification = 'center')]])]], justification = 'center')
        
        #Activities
        col3 = sg.Column([[sg.Frame('',
                        [[sg.Button('Get data', key = '-GET-DATA-', size = (25,5), font = 20, pad = (50,50)),
                            sg.Button('Build a deck', key = '-BUILD-DECK-', size = (25,5), font = 20, pad = (50,50))],
                        [sg.Button('Explore data', key = '-EXPLORE-DATA-', size = (25,5), font = 20, pad = (50,50)),
                            sg.Button('Pick/ban\n advisor', key = '-PICK-BAN-', size = (25,5), font = 20, pad = (50,50))]])]],
                                 justification = 'center')
        #Options and help
        tech_col = sg.Column([[sg.Text('\n\n')],
                [sg.Column([[sg.Button('Settings', key = '-SETTINGS-'),
                             sg.Button('Help', key = '-HELP1-'),
                             sg.Button('Quit')]],
                                        size=(155,45))]], justification = 'right') 

        layout = [[col1],
                  [col2],
                  [col3],
                  [tech_col]]
        
        sg.theme('LightGrey1')
        sg.set_options(font=("Arial", 10))
        
        return sg.Window('Hearthstone Archmage', layout, size = (1080, 820), finalize=True)
    
    def open_get_data_w(self):
        col1 = sg.Column([[sg.Frame('', [[sg.Text('Data Extraction', size = (60,1),
                                        font = ('Courier, 25'), justification = 'center')]])]], justification = 'center')
        
        #Driver and folder selection
        col2 = sg.Column([[sg.Frame('',
                                   [[sg.Text('Select the driver:', size = (24,1)),
                                     sg.Button('Here', key = '-DRIVER-PATH-'),
                                     sg.Text('-', size=(3,1), key='-DRIVER-PATH-OK-', justification = 'center')],
                                    [sg.Text('Select a folder for data storage:', size = (24,1)),
                                     sg.Button('Here', key = '-DECK-FOLDER-'),
                                     sg.Text('-', size=(3,1), key='-DECK-FOLDER-OK-', justification = 'center')]])]]
                          ,justification = 'left')
        
        col3 = sg.Column([[sg.Frame(layout=[
    [sg.Text('Extract data for:', size = (26,1), justification = 'center')],
    [sg.Text('', size = (8,1)), sg.Radio('All', 'RADIO1', default=True, size=(10,1), key = '-EXTRACT-ALL-')],  
    [sg.Radio('Demon Hunter', 'RADIO1', size=(10,1), key = '-EXTRACT-DEMON-HUNTER-'),
     sg.Radio('Druid', 'RADIO1', size=(10,1), key = '-EXTRACT-DRUID-')],
    [sg.Radio('Hunter', 'RADIO1', size=(10,1), key = '-EXTRACT-HUNTER-'),
     sg.Radio('Mage', 'RADIO1', size=(10,1), key = '-EXTRACT-MAGE-')],
    [sg.Radio('Paladin', 'RADIO1', size=(10,1), key = '-EXTRACT-PALADIN-'),
     sg.Radio('Priest', 'RADIO1', size=(10,1), key = '-EXTRACT-PRIEST-')],
    [sg.Radio('Rogue', 'RADIO1', size=(10,1), key = '-EXTRACT-ROGUE-'),
     sg.Radio('Shaman', 'RADIO1', size=(10,1), key = '-EXTRACT-SHAMAN-')],
    [sg.Radio('Warlock', 'RADIO1', size=(10,1), key = '-EXTRACT-WARLOCK-'),
     sg.Radio('Warrior', 'RADIO1', size=(10,1), key = '-EXTRACT-WARRIOR-')]],
                                   title='',title_color='black')]], justification = 'left')
        
        
        col4 = sg.Column([[sg.Frame(layout = [
    [sg.Text('Specify archetype:'),
     sg.Radio('Yes', 'archetype_selection', enable_events = True, key = '-SELECT-ARCHETYPE-YES-'),
     sg.Radio('No', 'archetype_selection', default = True, enable_events = True, key = '-SELECT-ARCHETYPE-NO-')],       
     [sg.Column([[sg.Text('', size = (5,3))]]
                , visible=True, key='-SELECT-ARCHETYPE-0-'),
      sg.Column([[sg.Text('Name:', size = (5,1)),
                 sg.I(size=(17, 1), key = '-EXTRACT-ARCHETYPE-'), sg.Submit('OK', size = (3,1), key = 'OK')],
                 [sg.Text('', size = (27,1), key = '-ARCHETYPE-CONFIRMATION-')]]
                , visible = False, key = '-SELECT-ARCHETYPE-1-')]],
                                   title = '')]], justification = 'left')
        
        
        tech_col = sg.Column([[sg.Text('\n\n')],
                [sg.Column([[sg.Button('Settings', key = '-SETTINGS-'),
                             sg.Button('Help', key = '-HELP2-'),
                             sg.Button('Back', key = '-BACK-')]],
                                        size=(165,45))]], justification = 'right')
        
        layout = [[col1],
                  [col2],
                  [col3],
                  [col4],
                 [tech_col]]
        
        #sg.theme('DarkBlue5')
        
        return sg.Window('Data extraction', layout, size = (1080, 820), finalize=True)
    
    
    def open_build_deck_w(self):
        col1 = sg.Column([[sg.Frame('', [[sg.Text('Deck building', size = (60,1),
                                        font = ('Courier, 25'), justification = 'center')]])]], justification = 'center')
        
        tech_col = sg.Column([[sg.Text('\n\n')],
                [sg.Column([[sg.Button('Settings', key = '-SETTINGS-'),
                             sg.Button('Help', key = '-HELP3-'),
                             sg.Button('Back', key = '-BACK-')]],
                                        size=(165,45))]], justification = 'right')         
        layout = [[col1],
                 [tech_col]]
        
        return sg.Window('Deck building', layout, size = (1080, 820), finalize=True)
        
        
    def open_explore_data_w(self):
        col1 = sg.Column([[sg.Frame('', [[sg.Text('Data exploration', size = (60,1),
                                        font = ('Courier, 25'), justification = 'center')]])]], justification = 'center')
        
        #Folder selection
        col2 = sg.Column([[sg.Frame('',
                                   [[sg.Text('Select the folder with raw data:', size = (24,1)),
                                     sg.Button('Here', key = '-DECK-FOLDER-'),
                                     sg.Text('-', size=(3,1), key='-DECK-FOLDER-OK-', justification = 'center')],
                                     [sg.Text('Select a folder for output storage:', size = (24,1)),
                                     sg.Button('Here', key = '-ANALYSIS-PATH-'),
                                     sg.Text('-', size=(3,1), key='-ANALYSIS-PATH-OK-', justification = 'center')]])]]
                          ,justification = 'left')        

        tech_col = sg.Column([[sg.Text('\n\n')],
                [sg.Column([[sg.Button('Settings', key = '-SETTINGS-'),
                             sg.Button('Help', key = '-HELP4-'),
                             sg.Button('Back', key = '-BACK-')]],
                                        size=(165,45))]], justification = 'right')        
        layout = [[col1],
                  [col2],
                 [tech_col]]
        
        return sg.Window('Data exploration', layout, size = (1080, 820), finalize=True)
    
    
    def open_pick_ban_w(self):
        col1 = sg.Column([[sg.Frame('', [[sg.Text('Pick/ban advisor', size = (60,1),
                                        font = ('Courier, 25'), justification = 'center')]])]], justification = 'center')
        
        tech_col = sg.Column([[sg.Text('\n\n')],
                [sg.Column([[sg.Button('Settings', key = '-SETTINGS-'),
                             sg.Button('Help', key = '-HELP5-'),
                             sg.Button('Back', key = '-BACK-')]],
                                        size=(165,45))]], justification = 'right')
        layout = [[col1],
                 [tech_col]]
        
        return sg.Window('Pick/ban advisor', layout, size = (1080, 820), finalize=True)
    
    def get_help_window(self, key):
        if key == '-HELP1-':
            sg.popup('This help should help you understand how to use the pick/ban advisor', title = 'Help')
        elif key == '-HELP2-':
            sg.popup('This help should help you understand how to explore data efficiently', title = 'Help')        
        elif key == '-HELP3-':
            sg.popup('This help should help you understand how to build your deck using statistics', title = 'Help')    
        elif key == '-HELP4-':
            sg.popup('This help should help you understand how to extract data into your computer', title = 'Help')
        elif key == '-HELP5-':
            sg.popup('This help should help you understand how to navigate the application', title = 'Help')            
            
        return None
    
    def select_folder(self, key):
        if key == '-DECK-FOLDER-':
            deck_folder = sg.popup_get_folder('Select the deck folder', title = 'Select')

        elif key == '-ANALYSIS-PATH-':
            analysis_path = sg.popup_get_folder('Select the analysis path', title = 'Select')
            
        try:
            deck_folder
        except NameError:
            deck_folder = None
            
        try:
            analysis_path
        except NameError:
            analysis_path = None            
        
        return deck_folder, analysis_path
    
    def select_file(self, key):
        if key == '-DRIVER-PATH-':
            driver_path = sg.popup_get_file('Select the driver', title = 'Select')
            
        try:
            driver_path
        except NameError:
            driver_path = None
        
        return driver_path
        
        
    
    def analyze(self):
        
        class_names_list = ['Demon Hunter', 'Druid', 'Hunter', 'Mage', 'Paladin',
                       'Priest', 'Rogue', 'Shaman', 'Warlock', 'Warrior']
        help_list = ['-HELP1-', '-HELP2-', '-HELP3-', '-HELP4-', '-HELP5-']
        archetype_tag = ['-SELECT-ARCHETYPE-NO-', '-SELECT-ARCHETYPE-YES-']

        init_w, get_data_w, build_deck_w, explore_data_w, pick_ban_w = self.open_init_w(), None, None, None, None
        
        while True:
            window, event, values = sg.read_all_windows()
            print(event)
            
            #Window closure
            if event == sg.WINDOW_CLOSED or event == '-BACK-' or event == 'Quit':
                if window == get_data_w:    #Close get_data_w and marked as closed
                    get_data_w.close()
                    get_data_w = None  
                elif window == build_deck_w:    #Close build_deck_w and marked as closed
                    build_deck_w.close()
                    build_deck_w = None
                elif window == explore_data_w:    #Close explore_data_w and marked as closed
                    explore_data_w.close()
                    explore_data_w = None
                elif window == pick_ban_w:    #Close pick_ban_w and marked as closed
                    pick_ban_w.close()
                    pick_ban_w = None                        
                elif window == init_w:
                    if get_data_w != None:    
                        get_data_w.close()
                    elif build_deck_w != None:
                        build_deck_w.close()
                    elif explore_data_w != None:
                        explore_data_w.close()
                    elif pick_ban_w != None:
                        pick_ban_w.close()
                    window.close()    #Close all other open windows and then the initial window itself
                    break

            #Folder and file selection        
            elif event == '-DECK-FOLDER-':
                deck_folder = self.select_folder(event)[0]
                if deck_folder == None:
                    window['-DECK-FOLDER-OK-'].update('x')
                else:
                    window['-DECK-FOLDER-OK-'].update('OK!')
                   
            elif event == '-ANALYSIS-PATH-':
                analysis_path = self.select_folder(event)[1]
                if analysis_path == None:
                    window['-ANALYSIS-PATH-OK-'].update('x')
                else:
                    window['-ANALYSIS-PATH-OK-'].update('OK!')
                
            elif event == '-DRIVER-PATH-':
                driver_path = self.select_file(event)
                if driver_path == None:
                    window['-DRIVER-PATH-OK-'].update('x')
                else:
                    window['-DRIVER-PATH-OK-'].update('OK!')                
            
            #Settings
            elif event == '-SETTINGS-':
                pass #define this in case of need
            
            #Defining functions shared across windows
            elif event in help_list:
                self.get_help_window(event)
                
            #Functions for Get Data window
            elif event in archetype_tag:
                for i in range(2):
                    window[f'-SELECT-ARCHETYPE-{i}-'].update(visible=values[archetype_tag[i]])
                    
            elif event == 'OK' and values['-EXTRACT-ARCHETYPE-'] != None:
                archetype_name = values['-EXTRACT-ARCHETYPE-']
                window['-EXTRACT-ARCHETYPE-'].update('')
                window['-ARCHETYPE-CONFIRMATION-'].update(archetype_name + ' selected.')
            
            #Extra window opening
            elif event == '-GET-DATA-'and get_data_w == None:
                get_data_w = self.open_get_data_w()
            elif event == '-BUILD-DECK-'and build_deck_w == None:
                build_deck_w = self.open_build_deck_w()
            elif event == '-EXPLORE-DATA-' and explore_data_w == None:
                explore_data_w = self.open_explore_data_w()
            elif event == '-PICK-BAN-'and pick_ban_w == None:
                pick_ban_w = self.open_pick_ban_w()
                
            

            #    window['-OUTPUT-'].update(f'You enetered {values["-IN-"]}')
            #elif event == 'Erase':
            #    window['-OUTPUT-'].update('')
            #    window['-IN-'].update('')
            
        return None
        
GA = GraphicalArchmage()        
GA.analyze()        

-GET-DATA-
-SELECT-ARCHETYPE-YES-
OK
-DRIVER-PATH-
-DECK-FOLDER-
None
None


In [25]:
layout = [[sg.Text('My one-shot window.')],      
                 [sg.InputText(key='-IN-')],      
                 [sg.Submit(), sg.Cancel()]]      

window = sg.Window('Window Title', layout)    

event, values = window.read()
print(event)
window.close()

text_input = values['-IN-']
sg.popup('You entered', text_input)

Submit


'OK'

In [10]:
import PySimpleGUI as sg
radio_tag = ["Next", "PREV"]

sg.theme("DarkBlue3")

cols = [
    [[sg.Text("Next element is sg.Input", size=(28, 1)), sg.Input()]],
    [[sg.Input(), sg.Text("Previous element is sg.Input", size=(28, 1))]],
]

layout =[
    [sg.Radio(text, "position", default=(i==0), enable_events=True, key=text)
        for i, text in enumerate(radio_tag)],
    [sg.Column(cols[i], visible=(i==0), key=f'COL {i}') for i in range(2)],
]

window = sg.Window("Title", layout, finalize=True)

while True:

    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break
    elif event in radio_tag:
        for i in range(2):
            window[f'COL {i}'].update(visible=values[radio_tag[i]])

window.close()

In [164]:
dir(sg)

['ALTERNATE_TABLE_AND_TREE_SELECTED_ROW_COLORS',
 'AddMenuItem',
 'AddToReturnDictionary',
 'AddToReturnList',
 'Any',
 'B',
 'BLUES',
 'BM',
 'BMenu',
 'BROWSE_FILES_DELIMITER',
 'BUTTON_DISABLED_MEANS_IGNORE',
 'BUTTON_TYPE_BROWSE_FILE',
 'BUTTON_TYPE_BROWSE_FILES',
 'BUTTON_TYPE_BROWSE_FOLDER',
 'BUTTON_TYPE_CALENDAR_CHOOSER',
 'BUTTON_TYPE_CLOSES_WIN',
 'BUTTON_TYPE_CLOSES_WIN_ONLY',
 'BUTTON_TYPE_COLOR_CHOOSER',
 'BUTTON_TYPE_READ_FORM',
 'BUTTON_TYPE_REALTIME',
 'BUTTON_TYPE_SAVEAS_FILE',
 'BUTTON_TYPE_SHOW_DEBUGGER',
 'Btn',
 'Button',
 'ButtonMenu',
 'CB',
 'CBox',
 'CButton',
 'COLOR_SCHEME',
 'COLOR_SYSTEM_DEFAULT',
 'CPRINT_DESTINATION_MULTILINE_ELMENT_KEY',
 'CPRINT_DESTINATION_WINDOW',
 'CURRENT_LOOK_AND_FEEL',
 'CUSTOM_MENUBAR_METADATA_MARKER',
 'CUSTOM_TITLEBAR_BACKGROUND_COLOR',
 'CUSTOM_TITLEBAR_FONT',
 'CUSTOM_TITLEBAR_ICON',
 'CUSTOM_TITLEBAR_TEXT_COLOR',
 'CalendarButton',
 'Cancel',
 'Canvas',
 'ChangeLookAndFeel',
 'Check',
 'Checkbox',
 'CloseButton',
 'Col',
 'C

In [156]:

def open_window():
    layout = [[sg.Text("New Window", key="new")]]
    window = sg.Window("Second Window", layout, modal=True)
    choice = None
    while True:
        event, values = window.read()
        if event == "Exit" or event == sg.WIN_CLOSED:
            break
        
    window.close()
def main():
    layout = [[sg.Button("Open Window", key="open")]]
    window = sg.Window("Main Window", layout)
    while True:
        event, values = window.read()
        if event == "Exit" or event == sg.WIN_CLOSED:
            break
        if event == "open":
            open_window()
        
    window.close()
if __name__ == "__main__":
    main()

In [59]:
help(sg.Button)

In [ ]:
#[sg.Checkbox('Checkbox', size=(10,1)),  sg.Checkbox('My second checkbox!', default=True)]

In [88]:
dir(sg)

['ALTERNATE_TABLE_AND_TREE_SELECTED_ROW_COLORS',
 'AddMenuItem',
 'AddToReturnDictionary',
 'AddToReturnList',
 'Any',
 'B',
 'BLUES',
 'BM',
 'BMenu',
 'BROWSE_FILES_DELIMITER',
 'BUTTON_DISABLED_MEANS_IGNORE',
 'BUTTON_TYPE_BROWSE_FILE',
 'BUTTON_TYPE_BROWSE_FILES',
 'BUTTON_TYPE_BROWSE_FOLDER',
 'BUTTON_TYPE_CALENDAR_CHOOSER',
 'BUTTON_TYPE_CLOSES_WIN',
 'BUTTON_TYPE_CLOSES_WIN_ONLY',
 'BUTTON_TYPE_COLOR_CHOOSER',
 'BUTTON_TYPE_READ_FORM',
 'BUTTON_TYPE_REALTIME',
 'BUTTON_TYPE_SAVEAS_FILE',
 'BUTTON_TYPE_SHOW_DEBUGGER',
 'Btn',
 'Button',
 'ButtonMenu',
 'CB',
 'CBox',
 'CButton',
 'COLOR_SCHEME',
 'COLOR_SYSTEM_DEFAULT',
 'CPRINT_DESTINATION_MULTILINE_ELMENT_KEY',
 'CPRINT_DESTINATION_WINDOW',
 'CURRENT_LOOK_AND_FEEL',
 'CUSTOM_MENUBAR_METADATA_MARKER',
 'CUSTOM_TITLEBAR_BACKGROUND_COLOR',
 'CUSTOM_TITLEBAR_FONT',
 'CUSTOM_TITLEBAR_ICON',
 'CUSTOM_TITLEBAR_TEXT_COLOR',
 'CalendarButton',
 'Cancel',
 'Canvas',
 'ChangeLookAndFeel',
 'Check',
 'Checkbox',
 'CloseButton',
 'Col',
 'C

In [8]:
#A simple window

# Define the window's contents
layout = [  [sg.Text("What's your name?")],     # Part 2 - The Layout
            [sg.Input()],
            [sg.Button('Ok')] ]

# Create the window
window = sg.Window('Window Title', layout)      # Part 3 - Window Defintion
                                                
# Display and interact with the Window
event, values = window.read()                   # Part 4 - Event loop or Window.read call

# Do something with the information gathered
print('Hello', values[0], "! Thanks for trying PySimpleGUI")

# Finish up by removing from the screen
window.close()                                  # Part 5 - Close the Window

Hello Peter ! Thanks for trying PySimpleGUI


In [9]:
some_object = 1

In [5]:
#A persistent window
# Define the window's contents
layout = [[sg.Text("What's your name?")],
          [sg.Input(key='-INPUT-')],
          [sg.Text(size=(40,1), key='-OUTPUT-')],
          [sg.Button('Ok'), sg.Button('Quit')]]

# Create the window
window = sg.Window('Window Title', layout)

# Display and interact with the Window using an Event Loop
while True:
    event, values = window.read()
    # See if user wants to quit or window was closed
    if event == sg.WINDOW_CLOSED or event == 'Quit':
        break
    # Output a message to the window
    window['-OUTPUT-'].update('Hello ' + values['-INPUT-'] + "! Thanks for trying PySimpleGUI")

# Finish up by removing from the screen
window.close()

In [49]:
layout = [[sg.Button(f'{row}, {col}') for col in range(4)] for row in range(4)]

event, values = sg.Window('List Comprehensions', layout).read(close=True)
#Use .read(close=True) appended to an sg.Window to close the window with a click

In [ ]:
#A compressed code to procude a full-fledged GUI
event, values = sg.Window('Window Title', [[sg.T("What's your name?")],[sg.I()],[sg.B('Ok')]]).read(close=True)

In [ ]:
#Radio elements
layout =  [
    [sg.Radio('My first Radio!', "RADIO1", default=True),
    sg.Radio('My second radio!', "RADIO1")]
]

In [ ]:
#Checkbox elements
layout =  [[sg.Checkbox('My first Checkbox!', default=True), sg.Checkbox('My second Checkbox!')]]